In [1]:
import geopandas as gpd
from preproceso import preprocesa

denue = gpd.read_file("datos/DENUE_INEGI_09_.shp")
agebs = gpd.read_file("datos/ageb_urb.shp")
usos_suleo = preprocesa(denue, agebs)
usos_suelo.head()

ImportError: No module named 'geopandas'

In [2]:
import geopandas as gpd

ImportError: No module named 'geopandas'